In [1]:
from pprint import pprint
from rdflib.plugins.sparql.parser import parseQuery
from rdflib.term import Variable, URIRef, Literal
import math

In [2]:
query = """
PREFIX pastec: <http://service.swissartresearch.net/pastec/> 
PREFIX crm: <http://www.cidoc-crm.org/cidoc/crm>
SELECT ?candidate WHERE 
{ 
    ?s pastec:firstPredicate pastec:firstObject .
      ?s  pastec:secondPredicate pastec:secondObject ;
          pastec:thirdPredicate pastec:thirdObject ;
          pastec:fourthPredicate pastec:fourthObject , pastec:fifthObject .
      <http://example.org/image.jpg> pastec:hasSimilarImage ?candidate }
"""

In [3]:
def getPrefixes(parsedQuery):
    prefixesRaw = parsedQuery[0]
    prefixes = {}
    for d in prefixesRaw:
        prefixes[d['prefix']] = str(d['iri'])
    return prefixes

def getTypeOfTriple(triple):
    if type(triple) is Variable:
        return Variable
    if 'string' in triple:
        return Literal
    else:
        return URIRef
    
def getValueOfTriple(triple, prefixes):
    t = getTypeOfTriple(triple)
    if t is Variable:
        return str(triple)
    if t is URIRef:
        try:
            part = triple['part'][0]['part'][0]['part']
        except:
            part = triple
        if 'prefix' in part:
            prefix = prefixes[part['prefix']]
            localname = part['localname']
            return prefix + localname
        else:
            return str(part)
    if t is Literal:
        return str(triple['string'])
    return False

def parseTriples(parsedQuery):
    prefixes = getPrefixes(parsedQuery)
    triples = []
    for d in parsedQuery.asList():
        if 'where' in d:
            for item in d['where'].items():
                iteration = 0
                prevSet = 0
                prevIteration = 0
                iteration = prevIteration
                triple = {}
                for s, tripleSet in enumerate(item[1][0]['triples']):
                    for i, statement in enumerate(tripleSet):
                        if iteration != prevIteration or s != prevSet:
                            triples.append(triple)
                            triple = {}
                            prevSet = s
                            prevIteration = iteration
                        iteration = math.floor(i/3)
                        if i%3 == 0:
                            key = "s"
                        elif i%3 == 1:
                            key = "p"
                        elif i%3 == 2:
                            key = "o"
                        
                        triple[key] =  {
                            "type": getTypeOfTriple(statement),
                            "value": getValueOfTriple(statement, prefixes)
                        }
                triples.append(triple)
    # For statements missing a subject, take the subject from the previous statement.
    # Afterwards remove statements missing a predicate and object. These artefacts stem 
    # from tense notation.
    for i, triple in enumerate(triples):
        if not 's' in triple:
            if i > 0 and 's' in triples[i-1]:
                triple['s'] = triples[i-1]['s']
    return [d for d in triples if 's' in d and 'p' in d and 'o' in d]

def getProjectionVariables(parsedQuery):
    variables = []
    for d in parsedQuery.asList():
        if 'projection' in d:
            for var in d['projection']:
                variables.append(str(var['var']))
    return variables

def parsePastecQuery(query):
    q = parseQuery(query)
    return {
        "variables": getProjectionVariables(q),
        "triples": parseTriples(q)
    }

result = parsePastecQuery(query)

pprint(result)

{'triples': [{'o': {'type': <class 'rdflib.term.URIRef'>,
                    'value': 'http://service.swissartresearch.net/pastec/firstObject'},
              'p': {'type': <class 'rdflib.term.URIRef'>,
                    'value': 'http://service.swissartresearch.net/pastec/firstPredicate'},
              's': {'type': <class 'rdflib.term.Variable'>, 'value': 's'}},
             {'o': {'type': <class 'rdflib.term.URIRef'>,
                    'value': 'http://service.swissartresearch.net/pastec/secondObject'},
              'p': {'type': <class 'rdflib.term.URIRef'>,
                    'value': 'http://service.swissartresearch.net/pastec/secondPredicate'},
              's': {'type': <class 'rdflib.term.Variable'>, 'value': 's'}},
             {'o': {'type': <class 'rdflib.term.URIRef'>,
                    'value': 'http://service.swissartresearch.net/pastec/thirdObject'},
              'p': {'type': <class 'rdflib.term.URIRef'>,
                    'value': 'http://service.swissar

In [4]:
for triple in result['triples']:
    if triple['p']['value'] == 'http://service.swissartresearch.net/pastec/hasSimilarImage':
        if triple['o']['value'] in result['variables']:
            print("Projecting similar images for", triple['s']['value'], "on",  triple['o']['value'])

Projecting similar images for http://example.org/image.jpg on candidate
